In [7]:
# -*- coding: utf-8 -*-
import pygame
import math
# (ลบ import random ออก)

# --- ค่าคงที่ของเกม (จำเป็นต้องกำหนด) ---
WIDTH, HEIGHT = 800, 600
BLOCK_SIZE = 20  # ขนาดของบล็อกต่อตัวอักษร
BACKGROUND_COLOR = (10, 20, 40)  # สีพื้นหลัง (น้ำเงินเข้ม)
WATER_COLOR = (20, 80, 200, 200)  # สีน้ำ (มีค่า alpha)
SURFACE_HIGHLIGHT = (100, 150, 255, 100) # สีคลื่นผิวน้ำ

# --- คลาสที่ 2: Water (จากโค้ดของคุณ) ---
WATER_RISE_PIXELS_PER_LETTER = BLOCK_SIZE  # น้ำจะขึ้นตามจำนวนตัวอักษรที่พิมพ์ได้
WATER_ANIM_DURATION = 0.9  # ระยะเวลาเคลื่อนขึ้นของน้ำ
WATER_SURFACE_WAVE_AMPLITUDE = 6
WATER_SURFACE_LENGTH = 140 # ความยาวคลื่นผิวน้ำ

# (ลบค่า FIXED_WATER_RISE_LETTERS และ FIXED_BLOCK_RISE_LETTERS ออก)


# --- คลาสที่ 1: typingText (แก้ไขให้สมบูรณ์) ---
class typingText:
    def __init__(self, screen, font, question, valid_answers):
        self.screen = screen
        self.font = font
        self.question = question
        self.valid_answers = set(ans.lower() for ans in valid_answers)
        self.user_input = ""
        self.message = ""
        self.message_color = (255, 255, 255)

    def handle_event(self, event):
        submitted = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_BACKSPACE:
                self.user_input = self.user_input[:-1]
            elif event.key == pygame.K_RETURN:
                if self.user_input:
                    submitted = True
            else:
                char = event.unicode
                if char.isprintable():
                    self.user_input += char
        return submitted

    def check_answer(self):
        is_correct = self.user_input.lower() in self.valid_answers
        if is_correct:
            self.message = "Correct!"
            self.message_color = (0, 255, 0)
        else:
            self.message = "Invalid or Unknown Word!"
            self.message_color = (255, 0, 0)
        return is_correct

    def get_answer_length(self):
        return len(self.user_input)

    def clear_input(self):
        self.user_input = ""
        self.message = "" 

    def draw(self): 
        question_surf = self.font.render(f"Question: {self.question}", True, (255, 255, 255))
        self.screen.blit(question_surf, (20, 20))

        input_rect = pygame.Rect(20, 60, WIDTH - 40, 50)
        pygame.draw.rect(self.screen, (255, 255, 255), input_rect, 2) 
        
        input_surf = self.font.render(self.user_input, True, (255, 255, 0))
        self.screen.blit(input_surf, (input_rect.x + 10, input_rect.y + 10))

        result_surf = self.font.render(self.message, True, self.message_color)
        self.screen.blit(result_surf, (20, 120))


class Water:
    def __init__(self, screen_h):
        self.screen_h = screen_h
        self.level_y = float(screen_h)
        self.visible = False
        self.animating = False
        self.start_y = self.level_y 
        self.target_y = self.level_y
        self.anim_time = 0.0
        self.anim_dur = WATER_ANIM_DURATION

    def start_rise(self, rise_pixels):
        if not self.visible:
            self.visible = True
        self.start_y = float(self.level_y)
        self.target_y = max(0.0, self.start_y - rise_pixels) # น้ำสามารถขึ้นไปจนถึง 0.0
        self.anim_time = 0.0
        self.animating = True

    def update(self, dt):
        if self.animating:
            self.anim_time += dt
            t = min(1.0, self.anim_time / self.anim_dur)
            t_ease = t * t * (3 - 2 * t)
            self.level_y = self.start_y + (self.target_y - self.start_y) * t_ease
            if t >= 1.0:
                self.level_y = self.target_y
                self.animating = False

    def draw(self, surf):
        if not self.visible:
            return
        level = int(self.level_y)
        if level < self.screen_h:
            water_surface = pygame.Surface((WIDTH, self.screen_h - level), pygame.SRCALPHA)
            water_surface.fill(WATER_COLOR)
            surf.blit(water_surface, (0, level))

            points = []
            t = pygame.time.get_ticks() / 1000.0
            amp = WATER_SURFACE_WAVE_AMPLITUDE
            length = WATER_SURFACE_LENGTH
            for x in range(0, WIDTH + 8, 8):
                y = level + math.sin((x / length) + t * 1.5) * amp
                points.append((x, y))
            if points:
                pygame.draw.lines(surf, (255, 255, 255), False, points, 3)

    def is_animating(self):
        return self.animating

    def reset(self):
        self.__init__(self.screen_h)


# --- คลาสที่ 3: Scoring ---
class Scoring:
    def __init__(self):
        self.char_score = 0
        self.word_score = 0
        self.words_history = []
        self.is_game_over = False

    def register_word(self, word: str):
        if self.is_game_over:
            return
        clean_word = word.strip()
        if not clean_word:
            return
        self.char_score += len(clean_word)
        self.word_score += 1
        self.words_history.append(clean_word)

    def game_over(self):
        self.is_game_over = True
        return self.get_final_score()

    def get_final_score(self):
        total = self.char_score + self.word_score
        return {
            "char_score": self.char_score,
            "word_score": self.word_score,
            "total_score": total,
            "words_history": self.words_history,
        }

    def reset(self):
        self.char_score = 0
        self.word_score = 0
        self.words_history = []
        self.is_game_over = False

# --- คลาสใหม่: Player ---
class Player:
    def __init__(self, screen_w, screen_h, block_size):
        self.screen_w = screen_w
        self.screen_h = screen_h
        self.block_size = block_size
        self.width = 100
        self.x_pos = (self.screen_w // 2) - (self.width // 2)
        self.color = (200, 200, 200)
        self.survivor_color = (255, 0, 0)
        self.total_height_pixels = self.block_size
        self.y_pos = self.screen_h - self.total_height_pixels

    def add_height(self, num_letters):
        """
        เพิ่มความสูงของบล็อกตามจำนวนตัวอักษร
        """
        self.total_height_pixels += num_letters * self.block_size
        # จำกัดไม่ให้บล็อกสูงเกินจอ (เผื่อไว้) แต่ให้ y_pos เป็น 0 ได้
        self.y_pos = self.screen_h - self.total_height_pixels

    def get_top_y(self):
        """
        คืนค่าพิกัด Y ของ "หัว" ผู้เล่น
        """
        # y_pos คือฐานของตัว survivor, ลบ 20 คือหัว
        return (self.screen_h - self.total_height_pixels) - 20 

    def draw(self, surf):
        # คำนวณ y_pos ใหม่ทุกครั้งที่วาด เพื่อให้ถูกต้อง
        self.y_pos = self.screen_h - self.total_height_pixels
        
        # วาดกองบล็อก
        # จำกัดไม่ให้บล็อกวาดทะลุเพดาน
        draw_height = min(self.total_height_pixels, self.screen_h)
        draw_y = self.screen_h - draw_height
        
        player_rect = pygame.Rect(self.x_pos, draw_y, self.width, draw_height)
        pygame.draw.rect(surf, self.color, player_rect)
        
        # วาด "survivor" (สี่เหลี่ยมสีแดง) บนสุด
        survivor_y = self.get_top_y()
        survivor_rect = pygame.Rect(self.x_pos + (self.width // 2) - 10, survivor_y, 20, 20)
        pygame.draw.rect(surf, self.survivor_color, survivor_rect)

    def reset(self):
        self.__init__(self.screen_w, self.screen_h, self.block_size)

# --- ฟังก์ชันหลัก (Main Game) ---
def main():
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Text or Die - Demo")
    
    try:
        main_font = pygame.font.Font(None, 40)
        score_font = pygame.font.Font(None, 50)
        small_font = pygame.font.Font(None, 24)
    except:
        main_font = pygame.font.SysFont(None, 40)
        score_font = pygame.font.SysFont(None, 50)
        small_font = pygame.font.SysFont(None, 24)

    clock = pygame.time.Clock()

    # --- ตั้งค่าเกม ---
    valid_answers = [
        "apple", "banana", "orange", "grape", "mango", "melon", "kiwi",
        "pear", "peach", "plum", "cherry", "berry", "lime", "lemon"
    ]
    question = "Name a fruit"

    player = Player(WIDTH, HEIGHT, BLOCK_SIZE)
    water = Water(HEIGHT)
    scoring = Scoring()
    typer = typingText(screen, main_font, question, valid_answers)

    running = True
    game_over = False
    game_won = False # <-- เพิ่มตัวแปรสถานะ "ชนะ"
    final_score_data = {}

    # --- Game Loop ---
    while running:
        dt = clock.tick(60) / 1000.0

        # --- 1. จัดการ Events ---
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            
            if game_over:
                if event.type == pygame.KEYDOWN and event.key == pygame.K_r:
                    player.reset()
                    water.reset()
                    scoring.reset()
                    typer.clear_input()
                    game_over = False
                    game_won = False # <-- รีเซ็ตสถานะชนะ
                    final_score_data = {}
            else:
                if not water.is_animating():
                    submitted = typer.handle_event(event)
                    
                    # ===== บล็อกตรรกะที่แก้ไขใหม่ (Dynamic Rise) =====
                    if submitted:
                        is_correct = typer.check_answer()
                        letters = typer.get_answer_length() # ความยาวคำที่พิมพ์
                        water_rise_letters = 0.0 # เริ่มต้น

                        if is_correct:
                            # 1. ตอบถูก: เก็บคะแนน
                            scoring.register_word(typer.user_input)
                            # 2. ตอบถูก: บล็อกสูงขึ้น = ความยาวเต็ม
                            player.add_height(letters)
                            # 3. ตอบถูก: น้ำสูงขึ้น = ครึ่งหนึ่งของความยาว
                            water_rise_letters = letters / 2.0
                            
                            # --- 2.1 ตรวจสอบการชนะ (ชนเพดาน) ---
                            if player.get_top_y() <= 0:
                                game_over = True
                                game_won = True
                                final_score_data = scoring.game_over()
                                # หยุดการทำงานใน loop นี้ (ไม่ต้องรอน้ำขึ้น)
                                typer.clear_input()
                                continue 
                        
                        else:
                            # 1. ตอบผิด: บล็อกไม่สูงขึ้น
                            # 2. ตอบผิด: น้ำสูงขึ้น = ครึ่งหนึ่งของความยาวคำที่ *พิมพ์ผิด*
                            water_rise_letters = letters / 2.0
                        
                        # สั่งให้น้ำขึ้น (ถ้าน้ำไม่หยุดนิ่ง และ เกมยังไม่ชนะ)
                        if water_rise_letters > 0:
                            rise_px = water_rise_letters * WATER_RISE_PIXELS_PER_LETTER
                            water.start_rise(rise_px)
                        
                        typer.clear_input()
                    # ===== จบส่วนที่แก้ไข =====

        # --- 2. อัปเดตสถานะเกม ---
        # (ต้องเช็คว่าเกมยังไม่จบจากการ "ชนะ" ก่อน)
        if not game_over:
            water.update(dt)
            
            # ตรวจสอบ Game Over (จมน้ำ)
            if not water.is_animating() and water.visible and water.level_y <= player.get_top_y():
                game_over = True
                game_won = False # ตั้งค่าว่า "แพ้" (จมน้ำ)
                final_score_data = scoring.game_over()

        # --- 3. วาดทุกอย่างลงจอ ---
        screen.fill(BACKGROUND_COLOR)
        player.draw(screen)
        water.draw(screen)
        
        if game_over:
            # --- วาดหน้าจอ Game Over / Win ---
            overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            overlay.fill((0, 0, 0, 180))
            screen.blit(overlay, (0, 0))
            
            # --- เปลี่ยนข้อความตามสถานะ ชนะ หรือ แพ้ ---
            if game_won:
                go_text = score_font.render("YOU REACHED THE TOP!", True, (0, 255, 100))
            else:
                go_text = score_font.render("GAME OVER", True, (255, 0, 0))
            
            screen.blit(go_text, (WIDTH // 2 - go_text.get_width() // 2, HEIGHT // 3))
            
            # (วาดคะแนนเหมือนเดิม)
            char_text = main_font.render(f"Character Score: {final_score_data.get('char_score', 0)}", True, (255, 255, 255))
            word_text = main_font.render(f"Word Score: {final_score_data.get('word_score', 0)}", True, (255, 255, 255))
            total_text = score_font.render(f"Total Score: {final_score_data.get('total_score', 0)}", True, (255, 255, 0))
            
            screen.blit(total_text, (WIDTH // 2 - total_text.get_width() // 2, HEIGHT // 2))
            screen.blit(char_text, (WIDTH // 2 - char_text.get_width() // 2, HEIGHT // 2 + 60))
            screen.blit(word_text, (WIDTH // 2 - word_text.get_width() // 2, HEIGHT // 2 + 100))
            
            restart_text = small_font.render("Press 'R' to Restart", True, (150, 150, 150))
            screen.blit(restart_text, (WIDTH // 2 - restart_text.get_width() // 2, HEIGHT - 50))
            
        else:
            typer.draw() 
            
        pygame.display.flip()

    pygame.quit()

# --- เริ่มเกม ---
if __name__ == "__main__":
    main()